In [2]:
from __future__ import annotations # allows multiple typing of arguments in Python versions prior to 3.10

In [3]:
# Code below included to ensure compatibility with scikit-learn v1.1.x
from sklearn import set_config
set_config(display='text')

In [4]:
#| include: false
from nbdev.showdoc import show_doc

In [5]:
import numpy as np
import pandas as pd
import pandas.api.types as ptypes
from gingado.model_documentation import ggdModelDocumentation, ModelCard
from gingado.utils import read_attr
from sklearn.base import BaseEstimator
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.utils.metaestimators import available_if
from sklearn.utils.validation import check_is_fitted

In [6]:
df_new = pd.read_csv("..\\data\\final\\proposed_dataset_21082023.csv")

In [7]:
unnamedCols = [col for col in df_new.columns if 'Unnamed' in col]
df_new.drop(columns=unnamedCols, inplace=True)

In [11]:
df_new.XX_MLA

0        86.898135
1        89.545334
2        90.997005
3        83.580184
4        80.905077
           ...    
20873    52.676231
20874    61.124212
20875    61.384902
20876    58.155715
20877    58.155715
Name: XX_MLA, Length: 20878, dtype: float64

In [12]:
df_new['CORE_DEPOSITS']=df_new.apply(lambda x: x['01_CURR_ACC']+x['03_SAVINGS']+x['04_OTHER_DEPOSITS'], axis=1)
df_new['TOTAL_DEPOSITS']=df_new.apply(lambda x: x['05_BANKS_DEPOSITS']+x['XX_CUSTOMER_DEPOSITS'], axis=1)
df_new['GL_TO_TOTAL_FUNDING']=df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['F125_LIAB_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_FUNDING']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F125_LIAB_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_ASSET']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['CD_TO_TOTAL_DEPOSIT']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['TOTAL_DEPOSITS']+0.0001), axis=1)
df_new['LiqAsset2DemandLiab']=df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['ExcessShortTLiab2LongTAsset']=df_new.apply(lambda x: (x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']-x['XX_TOTAL_LIQUID_ASSET'])/(x['F077_ASSETS_TOTAL'] - x['XX_TOTAL_LIQUID_ASSET']+0.0001), axis=1)
df_new['CD_TO_TOTAL_ASSET']=df_new.apply(lambda x: x['CORE_DEPOSITS']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['GL_TO_TOTAL_DEPOSITS']=df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['LIQASSET2TOTALASSET']=df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['F077_ASSETS_TOTAL'] +0.0001), axis=1)
df_new['BANKSIZE']= np.log10(df_new['F077_ASSETS_TOTAL'])
df_new['LOAN2DEPOSIT']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT'] +0.0001), axis=1)
df_new['LIQASSET2DEPOSIT']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['CORE_DEPOSITS'] + x['02_TIME_DEPOSIT']+0.0001), axis=1)
df_new['CURRENTRATIO']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['XX_TOTAL_LIQUID_LIAB'] + 0.0001), axis=1)
df_new['LIQASSET2TOTALASSET']= df_new.apply(lambda x: x['XX_TOTAL_LIQUID_ASSET']/(x['F077_ASSETS_TOTAL'] + 0.0001), axis=1)
df_new['VOLATILEDEPOSITS2LIAB']= df_new.apply(lambda x: (x['XX_TOTAL_LIQUID_ASSET']-x['CORE_DEPOSITS'])/(x['F125_LIAB_TOTAL'] + 0.0001), axis=1)
df_new['LOAN2ASSETS']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['DOMESTICDEPOSIT2ASSETS']= df_new.apply(lambda x: (x['TOTAL_DEPOSITS']-x['10_FOREIGN_DEPOSITS_AND_BORROWINGS'])/(x['F077_ASSETS_TOTAL']+0.0001), axis=1)
df_new['LOAN2COREDEPOSIT']= df_new.apply(lambda x: x['EWAQ_GrossLoans']/(x['CORE_DEPOSITS']+0.0001), axis=1)
df_new['BOTBAL2TOTALDEPOSIT']= df_new.apply(lambda x: x['XX_BOT_BALANCE']/(x['TOTAL_DEPOSITS']+0.0001), axis=1)



In [13]:
from statsmodels.tsa.arima.model import ARIMA


In [14]:
X = df_new[[
'REPORTINGDATE',    
'INSTITUTIONCODE',    
'BANKSIZE', 
'LR',
'EWAQ_NPLsNetOfProvisions',  
'EWAQ_NPLsNetOfProvisions2CoreCapital', #little contribution
'EWAQ_NPL', 
'ExcessShortTLiab2LongTAsset',
'CD_TO_TOTAL_ASSET',
'LIQASSET2TOTALASSET',  
'LIQASSET2DEPOSIT', 
'TOTAL_DEPOSITS'
]]

#model target/label
Y = df_new[['XX_MLA']]

In [29]:
#without bank codes and reporting date
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(scaled_X,Y,test_size=0.33,random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X.values,Y.T.values[0],test_size=0.33,random_state=42)

In [19]:
Y = df_new[df_new.INSTITUTIONCODE=='B5014'][['REPORTINGDATE','INSTITUTIONCODE','XX_MLA']].set_index('REPORTINGDATE')
Y

,INSTITUTIONCODE,XX_MLA
REPORTINGDATE,,
2010-02-05,B5014,86.898135
2010-02-12,B5014,89.545334
2010-02-19,B5014,90.997005
2010-02-26,B5014,83.580184
2010-03-05,B5014,80.905077
...,...,...
2021-12-10,B5014,31.833337
2021-12-17,B5014,31.651096
2021-12-24,B5014,34.392871


In [ ]:
def arima(data)
    y = data[['XX_MLA']]
    y_test = y[y.index >= '2020-01-01']
    y_train = y[y.index < '2020-01-01']
    model = ARIMA(train2, order=(1,1,50))
    model_fit = model.fit()
    print(model_fit.summary())

In [20]:
y = Y[['XX_MLA']].sort_index()
y_test = y[y.index >= '2020-01-01']
y_test

,XX_MLA
REPORTINGDATE,
2020-01-03,42.762402
2020-01-10,43.869761
2020-01-17,38.219929
2020-01-24,39.615658
2020-01-31,39.994127
...,...
2021-12-10,31.833337
2021-12-17,31.651096
2021-12-24,34.392871


In [21]:
y_train = y[y.index < '2020-01-01']
y_train

,XX_MLA
REPORTINGDATE,
2010-02-05,86.898135
2010-02-12,89.545334
2010-02-19,90.997005
2010-02-26,83.580184
2010-03-05,80.905077
...,...
2019-11-29,44.521018
2019-12-06,42.738920
2019-12-13,41.805232


In [22]:
model = ARIMA(y_train, order=(1,1,1))
model_fit = model.fit()
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                 XX_MLA   No. Observations:                  533
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -2284.380
Date:                Wed, 02 Oct 2024   AIC                           4574.761
Time:                        11:09:46   BIC                           4587.591
Sample:                             0   HQIC                          4579.782
                                - 533                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.5708      0.031    -18.637      0.000      -0.631      -0.511
ma.L1          0.1859      0.047      3.990      0.000       0.095       0.277
sigma2       314.0629      3.222     97.469      0.0

C:\Users\jqb_p\anaconda3\envs\bisih\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jqb_p\anaconda3\envs\bisih\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jqb_p\anaconda3\envs\bisih\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [23]:
# make predictions
predictions = model_fit.forecast(291)
print(f'ARIMA Model Test Data MSE: {np.mean((predictions.values - test2.values)**2):.3f}')

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'